In [1]:
import numpy as np
import math 
from sympy import *
from sympy.abc import x,y,X,Y
import random
import sys
import pandas as pd
import pickle
import plotly.express as px
# %load_ext memory_profiler
from sympy import Symbol 
from sympy import MatrixSymbol, Matrix
from collections import deque
PRECISION = 6

# Characterizing Uncertainty Set 

In [2]:
def bin_space_gen(d):
    all_coeff = {}
    for i in range(0,2**(d)): 
        t = i
        all_coeff[i] = ""
        for j in range (0,d):
            all_coeff[i]= str(t%2)+all_coeff[i]
            t = int(t/2)
    return all_coeff

def valid(vect,COUNT):
    cnt = 0
    for i in range(0,len(vect)):
        if vect[i] == "1":
            cnt+=1
    if(cnt == COUNT) :
        return 1 
    else :
        return 0
    
def count(a, b):
    m = len(a)
    n = len(b)
 
    lookup = [[0] * (n + 1) for i in range(m + 1)]
 
    # If first string is empty
    for i in range(n+1):
        lookup[0][i] = 0
 
    # If second string is empty
    for i in range(m + 1):
        lookup[i][0] = 1
 
    # Fill lookup[][] in bottom up manner
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if a[i - 1] == b[j - 1]:
                lookup[i][j] = lookup[i - 1][j - 1] + lookup[i - 1][j]     
            else:
                # If last character are different, ignore last character of first string
                lookup[i][j] = lookup[i - 1][j]
 
    return lookup[m][n]

def size_ins_ball(n_temp,m_temp):
    temp = 0
    for r in range(m_temp,n_temp+1):
        temp += math.comb(n_temp,r)
    return temp

In [4]:
n = 7
m = 4
sup_arr = bin_space_gen(n)
sub_arr = bin_space_gen(m)
X = MatrixSymbol('X', m+1, 2)

In [5]:
def choose(n, k):
    """
    A fast way to calculate binomial coefficients by Andrew Dalke (contrib).
    """
    if 0 <= k <= n:
        ntok = 1
        ktok = 1
        for t in range(1, min(k, n - k) + 1):
            ntok *= n
            ktok *= t
            n -= 1
        return ntok // ktok
    else:
        return 0

In [6]:
card_Y = 0
for i in range(0,n-m+1):
    card_Y += choose(n-1-i,m-1)*(2**(i))
card_Y

64

In [7]:
def add_node(nodes,parent_stack,s,n_temp, m_temp, parent):
    # print(s,n_temp, m_temp, parent)
    # print(nodes)
    # print(parent_stack)
    if(n_temp>0 and n_temp!=m_temp and m_temp>0):
        if(s=="l"):
            if(m_temp-1>=0): 
                parent_stack.append(len(parent_stack))
                nodes.append((len(nodes),n_temp,m_temp,parent,s))
                temp = nodes[len(nodes)-1][0]
                add_node(nodes,parent_stack,"l",n_temp-1, m_temp-1, temp)
                add_node(nodes,parent_stack,"r",n_temp-1, m_temp,temp)
                
        elif(s=="r"):
            parent_stack.append(len(parent_stack))
            nodes.append((len(nodes),n_temp,m_temp,parent,s))
            temp = nodes[len(nodes)-1][0]
            add_node(nodes,parent_stack,"l",n_temp-1, m_temp-1, temp)
            add_node(nodes,parent_stack,"r",n_temp-1, m_temp, temp)
        parent_stack.pop()
        
    elif(n_temp==m_temp or m_temp==0):
        nodes.append((len(nodes),n_temp,m_temp,parent,s))

In [8]:
cnt = 0 
nodes = deque([(0,n,m,-1,"root")])
parent_stack = deque([0])
n_temp = n
m_temp = m
parent = 0
    
while parent_stack:
    add_node(nodes,parent_stack,"l",n_temp-1, m_temp-1, parent)
    add_node(nodes,parent_stack,"r",n_temp-1, m_temp, parent)
    parent_stack.popleft()

In [9]:
# nodes

In [10]:
ins_ball = [[]]
cnt = 1 

while cnt <len(nodes):
    temp = nodes[cnt]
    parent = temp[3]
    if(temp[4]=="l"):
        ins_ball.append(ins_ball[parent]+[X[m-(nodes[parent][2])+1,0]])
    elif(temp[4]=="r"):
        ins_ball.append(ins_ball[parent]+[X[m-(nodes[parent][2])+1,1]])
    cnt+=1
    # print(ins_ball)

In [11]:
ins_ball_list = []
cnt = 0

while cnt<len(nodes):
    temp = nodes[cnt]
    n_temp = temp[1]
    m_temp = temp[2]
    if(m_temp==0):
        bin_arr = bin_space_gen(n_temp)
        for i in range(len(bin_arr)):
            ins_ball_list.append(ins_ball[cnt]+list(bin_arr[i]))
            
    elif(n_temp==m_temp):
        temp = []
        for i in range(m_temp):
            temp.append(X[m-m_temp+1+i,0])
        ins_ball_list.append(ins_ball[cnt]+temp)
    
    cnt+=1 
# print(ins_ball_list)

In [12]:
def list_to_str(lst):
    strn =""
    for i in range(len(lst)):
        strn = strn+str(lst[i])
    return strn

In [39]:
ins_ball_eval = {}
alpha = "11"
beta = ""
for s in range(0,int(len(sub_arr)/2)):
    print(s)
    ins_ball_eval[s] = {}
    sub_seq = sub_arr[s]
    for i in range(card_Y):
        ins_ball_eval[s][i] = [[],[]]
        for j in range(len(ins_ball_list[i])):
            temp = ins_ball_list[i][j]
            for k in range(1,m+1):
                if(type(temp)!= str):
                    temp = temp.subs(X[k,0],sub_seq[k-1])
                    temp = temp.subs(X[k,1],(int(sub_seq[k-1])+1)%2)
            ins_ball_eval[s][i][0].append(int(temp))
        ins_ball_eval[s][i][1].append(count(list_to_str(ins_ball_eval[s][i][0]),sub_seq))
        ins_ball_eval[s][i][1].append(count(alpha + list_to_str(ins_ball_eval[s][i][0])+beta,alpha + sub_seq+beta))
#         ins_ball_eval[s][i][2].append(ins_ball_list[i])

0
1
2
3
4
5
6
7


In [40]:
for s in range(0,int(len(sub_arr)/2)):
  print(sub_arr[s])
  for i in range(card_Y):
    if(ins_ball_eval[s][i][1][1]%ins_ball_eval[s][i][1][0] != 0 and ins_ball_eval[s][i][0][0] == int(sub_arr[s][0]) ):
      print(ins_ball_eval[s][i])

0000
[[0, 0, 1, 0, 0, 0, 0], [15, 17]]
[[0, 1, 0, 0, 0, 0, 0], [15, 25]]
[[0, 1, 0, 0, 0, 0, 1], [5, 7]]
[[0, 1, 0, 0, 0, 1, 0], [5, 7]]
[[0, 1, 0, 0, 1, 0, 0], [5, 7]]
[[0, 1, 0, 1, 0, 0, 0], [5, 7]]
0001
[[0, 0, 1, 0, 0, 0, 1], [10, 12]]
[[0, 1, 0, 0, 1, 0, 1], [5, 7]]
[[0, 1, 0, 0, 0, 1, 0], [4, 6]]
[[0, 1, 0, 0, 0, 1, 1], [8, 12]]
[[0, 1, 0, 0, 0, 0, 1], [10, 18]]
[[0, 1, 0, 1, 0, 0, 1], [4, 6]]
[[0, 1, 1, 0, 0, 0, 1], [4, 9]]
0010
[[0, 0, 1, 0, 0, 1, 0], [9, 11]]
[[0, 1, 0, 1, 0, 1, 0], [5, 7]]
[[0, 1, 0, 0, 1, 0, 0], [6, 10]]
[[0, 1, 0, 0, 1, 0, 1], [3, 5]]
[[0, 1, 0, 0, 1, 1, 0], [6, 10]]
[[0, 1, 1, 0, 0, 1, 0], [3, 8]]
0011
[[0, 0, 1, 0, 0, 1, 1], [8, 10]]
[[0, 1, 0, 1, 0, 1, 1], [5, 7]]
[[0, 1, 0, 0, 1, 1, 0], [3, 5]]
[[0, 1, 0, 0, 1, 1, 1], [9, 15]]
[[0, 1, 0, 0, 1, 0, 1], [3, 5]]
[[0, 1, 1, 0, 0, 1, 1], [3, 8]]
0100
[[0, 1, 0, 0, 1, 0, 0], [9, 13]]
[[0, 1, 0, 1, 0, 0, 0], [12, 18]]
[[0, 1, 0, 1, 0, 0, 1], [5, 7]]
[[0, 1, 0, 1, 0, 1, 0], [5, 7]]
[[0, 1, 0, 1, 1, 0, 0], [7, 11

In [52]:
s = 0
print(sub_arr[s])
for i in range(card_Y):
  if(ins_ball_eval[s][i][1][0] != ins_ball_eval[s][i][1][1] and ins_ball_eval[s][i][0][0] == int(sub_arr[s][0]) ):
    print(ins_ball_eval[s][i])

00000
[[0, 0, 0, 0, 0, 0, 0, 0, 0], [126, 210], [[X[1, 0], X[2, 0], X[3, 0], X[4, 0], X[5, 0], '0', '0', '0', '0']]]
[[0, 0, 0, 0, 0, 0, 0, 0, 1], [56, 84], [[X[1, 0], X[2, 0], X[3, 0], X[4, 0], X[5, 0], '0', '0', '0', '1']]]
[[0, 0, 0, 0, 0, 0, 0, 1, 0], [56, 84], [[X[1, 0], X[2, 0], X[3, 0], X[4, 0], X[5, 0], '0', '0', '1', '0']]]
[[0, 0, 0, 0, 0, 0, 0, 1, 1], [21, 28], [[X[1, 0], X[2, 0], X[3, 0], X[4, 0], X[5, 0], '0', '0', '1', '1']]]
[[0, 0, 0, 0, 0, 0, 1, 0, 0], [56, 84], [[X[1, 0], X[2, 0], X[3, 0], X[4, 0], X[5, 0], '0', '1', '0', '0']]]
[[0, 0, 0, 0, 0, 0, 1, 0, 1], [21, 28], [[X[1, 0], X[2, 0], X[3, 0], X[4, 0], X[5, 0], '0', '1', '0', '1']]]
[[0, 0, 0, 0, 0, 0, 1, 1, 0], [21, 28], [[X[1, 0], X[2, 0], X[3, 0], X[4, 0], X[5, 0], '0', '1', '1', '0']]]
[[0, 0, 0, 0, 0, 0, 1, 1, 1], [6, 7], [[X[1, 0], X[2, 0], X[3, 0], X[4, 0], X[5, 0], '0', '1', '1', '1']]]
[[0, 0, 0, 0, 0, 1, 0, 0, 0], [56, 84], [[X[1, 0], X[2, 0], X[3, 0], X[4, 0], X[5, 0], '1', '0', '0', '0']]]
[[0, 0, 0, 0,

In [35]:
def wlogw(wt):
    return wt*math.log(wt,2)

In [66]:
s = 1
print(sub_arr[s])

temp = 0

for i in range(card_Y):
  if(ins_ball_eval[s][i][0][0] != int(alpha) and ins_ball_eval[s][i][1][0] != ins_ball_eval[s][i][1][1]):  
    print(ins_ball_eval[s][i])
    temp += wlogw(ins_ball_eval[s][i][1][0])

print("\n")



00001
[[1, 0, 0, 0, 0, 1, 0, 0, 1], [16, 22], [[X[1, 1], X[1, 0], X[2, 0], X[3, 0], X[4, 0], X[5, 0], '0', '0', '1']]]
[[1, 0, 0, 0, 0, 1, 0, 1, 0], [6, 7], [[X[1, 1], X[1, 0], X[2, 0], X[3, 0], X[4, 0], X[5, 0], '0', '1', '0']]]
[[1, 0, 0, 0, 0, 1, 0, 1, 1], [11, 13], [[X[1, 1], X[1, 0], X[2, 0], X[3, 0], X[4, 0], X[5, 0], '0', '1', '1']]]
[[1, 0, 0, 0, 0, 1, 1, 0, 1], [7, 8], [[X[1, 1], X[1, 0], X[2, 0], X[3, 0], X[4, 0], X[5, 0], '1', '0', '1']]]
[[1, 0, 0, 0, 0, 0, 1, 0, 0], [5, 6], [[X[1, 1], X[1, 0], X[2, 0], X[3, 0], X[4, 0], X[5, 1], X[5, 0], '0', '0']]]
[[1, 0, 0, 0, 0, 0, 1, 0, 1], [20, 27], [[X[1, 1], X[1, 0], X[2, 0], X[3, 0], X[4, 0], X[5, 1], X[5, 0], '0', '1']]]
[[1, 0, 0, 0, 0, 0, 1, 1, 0], [10, 12], [[X[1, 1], X[1, 0], X[2, 0], X[3, 0], X[4, 0], X[5, 1], X[5, 0], '1', '0']]]
[[1, 0, 0, 0, 0, 0, 1, 1, 1], [15, 18], [[X[1, 1], X[1, 0], X[2, 0], X[3, 0], X[4, 0], X[5, 1], X[5, 0], '1', '1']]]
[[1, 0, 0, 0, 0, 0, 0, 1, 0], [15, 21], [[X[1, 1], X[1, 0], X[2, 0], X[3, 0], X[

In [63]:
temp

852.3051085590389